In [66]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [67]:
class RidgeRegression:
    def __init__(self, regulizer=1.0):
        self.regulizer = regulizer  # Regularization strength
        self.coffs = None  # Model Weights
        self.b = None  # b term

    def fit(self, X, y):
        n_samples, n_features = X.shape

        X_augmented = np.hstack([np.ones((n_samples, 1)), X])    # Augmentation (1's Column)

        I = np.eye(X_augmented.shape[1])
        I[0, 0] = 0  # for not regulizing b

        XtX = np.dot(X_augmented.T, X_augmented)  # XT * X
        XtY = np.dot(X_augmented.T, y)  # XT * y

        XtX_reg = XtX + self.regulizer * I      # XTX+λI

        XtX_reg_inv = np.linalg.inv(XtX_reg)    # (XTX+λI)^−1
        coffs = np.dot(XtX_reg_inv, XtY)        # W=(XTX+λI)^−1 XTy

        self.b = coffs[0]  # The first value is the b
        self.coffs = coffs[1:]  # The rest are the Weights


    def predict(self, X):
        #------np.dot(----------------Augmented X--------,-------b added to W in one vecto----)----
        return np.dot(np.hstack([np.ones((X.shape[0], 1)), X]), np.hstack([self.b, self.coffs]))   #y^=X⋅W

ridge_custom = RidgeRegression(regulizer=1.0)


In [68]:
ridge_custom.fit(X_train, y_train)

y_pred_custom = ridge_custom.predict(X_test)
mse_custom = mean_squared_error(y_test, y_pred_custom)
print("Custom Ridge Regression MSE:", mse_custom)

ridge_sklearn = Ridge(alpha=1.0)
ridge_sklearn.fit(X_train, y_train)
y_pred_sklearn = ridge_sklearn.predict(X_test)
mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)
print("Scikit-learn Ridge Regression MSE:", mse_sklearn)

print("Custom Coefficients:", ridge_custom.coffs)
print("Scikit-learn Coefficients:", ridge_sklearn.coef_)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_custom_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    ridge_custom = RidgeRegression(regulizer=1.0)
    ridge_custom.fit(X_train, y_train)
    y_pred_custom = ridge_custom.predict(X_test)

    mse_custom = mean_squared_error(y_test, y_pred_custom)
    mse_custom_scores.append(mse_custom)

mean_mse_custom = np.mean(mse_custom_scores)
print("Custom Ridge Regression Mean MSE with Cross-Validation:", mean_mse_custom)

mse_sklearn_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    ridge_sklearn = Ridge(alpha=1.0)
    ridge_sklearn.fit(X_train, y_train)
    y_pred_sklearn = ridge_sklearn.predict(X_test)

    mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)
    mse_sklearn_scores.append(mse_sklearn)

mean_mse_sklearn = np.mean(mse_sklearn_scores)
print("Scikit-learn Ridge Regression Mean MSE with Cross-Validation:", mean_mse_sklearn)


Custom Ridge Regression MSE: 3077.4159388272296
Scikit-learn Ridge Regression MSE: 3077.41593882723
Custom Coefficients: [  45.36737726  -76.66608563  291.33883165  198.99581745   -0.53030959
  -28.57704987 -144.51190505  119.26006559  230.22160832  112.14983004]
Scikit-learn Coefficients: [  45.36737726  -76.66608563  291.33883165  198.99581745   -0.53030959
  -28.57704987 -144.51190505  119.26006559  230.22160832  112.14983004]
Custom Ridge Regression Mean MSE with Cross-Validation: 3433.171835094969
Scikit-learn Ridge Regression Mean MSE with Cross-Validation: 3433.171835094969
